In [ ]:
FLOP = (2 * 1024 * 1600 * 1600 * 3  # QKVproj = 3*2Ld_m^2
+ 2 * 25 * 1024 * 1024 * 64 # QKT (bs,h,T,d_k) @ (bs,h,T,d_k) = 2hL^2d_k
+ 2 * 25 * 1024 * 1024 * 64 # Attn @ V (bs,h,T,T) @ (bs,h,T,d_k) = 2hL^2d_k
+ 2 * 1024 * 1600 * 1600 # Val @ Wo^T (bs,T,d_m) @ d_m, d_m = 2Ld_m^2
# + 2 * 1024 * 1600 * 6400 * 3 # Up L,d_m @ d_m,d_ff
) * 48 + 2 * 1024 * 1600 * 50256 + 2 * 1024 * 50256 * 1600 # final proj
print(FLOP)

27682406400


In [ ]:

def tokenize_text_to_binary(
    text_path: str,
    output_path: str,
    vocab_path: str = "data/TinyStories_train/vocab_str.json",
    merges_path: str = "data/TinyStories_train/merges_str.json",
    special_tokens: list[str] = ("<|endoftext|>",),
    dtype: np.dtype = np.int32,
) -> int:
    from cs336_basics.tokenizer import Tokenizer

    os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
    tokenizer = Tokenizer.from_files(vocab_path, merges_path, list(special_tokens))
    
    with open(text_path, "r", encoding="utf-8") as fin, open(output_path, "wb") as fout:
        tokens: Iterable[int] = tokenizer.encode_iterable(fin)
        
        # 使用tqdm显示进度条
        with tqdm(desc="标记化进度", unit="tokens", unit_scale=True) as pbar:
            for tok in tokens:
                # 将int转换为指定dtype的numpy数组再写入字节
                fout.write(np.array([tok], dtype=dtype).tobytes())
                pbar.update(1)

    return 

In [7]:
def cal_FLOP(layer=48, d_model=1600, heads=25, context_length=1024, d_ff=6400, vocab_size=50257):
    d_k = d_model // heads
    FLOP = (2 * context_length * d_model * d_model * 3  # QKVproj = 3*2Ld_m^2 
    + 2 * heads * context_length * context_length * d_k # QKT (bs,h,T,d_k) @ (bs,h,T,d_k) = 2hL^2d_k
    + 2 * heads * context_length * context_length * d_k # Attn @ V (bs,h,T,T) @ (bs,h,T,d_k) = 2hL^2d_k
    + 2 * context_length * d_model * d_model # Val @ Wo^T (bs,T,d_m) @ d_m, d_m = 2Ld_m^2
    + 2 * context_length * d_model * d_ff * 3 ) * layer + 2 * context_length * vocab_size * d_model  + 2 * context_length * d_model * vocab_size # final proj
    Attention = layer * (2 * context_length * d_model * d_model * 3 + 2 * heads * context_length * context_length * d_k
    + 2 * heads * context_length * context_length * d_k
    + 2 * context_length * d_model * d_model
    )
    FFN = layer * (2 * context_length * d_model * d_ff * 3)
    Projection = 2 * context_length * vocab_size * d_model  + 2 * context_length * d_model * vocab_size
    print("Attention: ", Attention, f"({Attention / FLOP:.2%})", "FFN: ", FFN, f"({FFN / FLOP:.2%})", "Projection: ", Projection, f"({Projection / FLOP:.2%})")
    return FLOP


print(cal_FLOP(layer=12, d_model=768, heads=12))
print(cal_FLOP(layer=24, d_model=1024, heads=16))
print(cal_FLOP(layer=36, d_model=1280, heads=20))
print(cal_FLOP())
print(cal_FLOP(context_length=16384))

Attention:  96636764160 (15.66%) FFN:  362387865600 (58.72%) Projection:  158094852096 (25.62%)
617119481856
Attention:  309237645312 (20.80%) FFN:  966367641600 (65.01%) Projection:  210793136128 (14.18%)
1486398423040
Attention:  676457349120 (24.58%) FFN:  1811939328000 (65.84%) Projection:  263491420160 (9.57%)
2751888097280
Attention:  1328755507200 (28.40%) FFN:  3019898880000 (64.56%) Projection:  329364275200 (7.04%)
4678018662400
Attention:  98569499443200 (64.78%) FFN:  48318382080000 (31.76%) Projection:  5269828403200 (3.46%)
152157709926400
